In [ ]:
from hdbcli import dbapi
import pandas as pd
import nltk as nltk
import seaborn as sns

In [ ]:
connection = dbapi.connect('52.214.219.42', 39015, 'SYSTEM', 'Glorp2018!')
connection.isconnected()

In [ ]:
cursor = connection.cursor()

# Import t3n Data into HANA

In [ ]:
t3n_data = pd.read_csv('../praktikum1/t3n_data_spider.csv')

In [ ]:
def create_table():
    sql = 'CREATE COLUMN TABLE \"SYSTEM\".\"T3N\" (\"ID\" VARCHAR(500),\"HEADING\" VARCHAR(2000),\"CATEGORY\" VARCHAR(2000),\"TEASER\" VARCHAR(5000),\"TEXT\" CLOB MEMORY THRESHOLD 1000,\"URL\" VARCHAR(2000), PRIMARY KEY (\"ID\"))'
    cursor.execute(sql)


In [ ]:
def get_escaped_string(string):
    return '\'' + str(string) + '\''

In [ ]:
def insert_data():
    for index, row in t3n_data.iterrows():
        sql_insert_datapoint = 'insert into \"SYSTEM\".\"T3N\" (ID, HEADING, CATEGORY, TEASER, TEXT, URL) VALUES (' + get_escaped_string(row['id']) + ', ' + get_escaped_string(row['heading']) +', ' + get_escaped_string(row['category']) + ',' + get_escaped_string(row['teaser']) + ', ' + get_escaped_string(row['text']) + ', ' + get_escaped_string(row['url'])+ ')'
        print(sql_insert_datapoint)
        #print(row['heading'])
        cursor.execute(sql_insert_datapoint)
    

# Fetch Textindex from HANA

In [ ]:
cmpl_data = pd.read_csv('./_TA_CDESCRIND__201911271735.csv')

In [ ]:
# Caution !! Takes ages ! 
cursor.execute('select * from "$TA_CDESCRIND"')
cursor.fetchall()

## Nouns per Document

In [ ]:
cmpl_nouns = cmpl_data[cmpl_data['TA_TYPE']=="noun"]
nouns_per_doc = cmpl_nouns["CMPLID"].value_counts()
ax = sns.distplot(nouns_per_doc)
ax.set(xlabel='#nouns per document')
ax.set_title('Nouns per document')
fig = ax.get_figure()

# Tokenization (obsolete)

In [ ]:
nltk.download('punkt')

In [ ]:
t3n_data_tokenized = t3n_data
t3n_data_tokenized['text'] = t3n_data_tokenized.apply(lambda row: nltk.word_tokenize(str(row['text'])), axis=1)
t3n_data_tokenized['teaser'] = t3n_data_tokenized.apply(lambda row: nltk.word_tokenize(str(row['teaser'])), axis=1)
t3n_data_tokenized['heading'] = t3n_data_tokenized.apply(lambda row: nltk.word_tokenize(str(row['heading'])), axis=1)

In [ ]:
for index, row in t3n_data.iterrows():
    print(row['heading'])